TODO : generaliser à images 32x32, etc.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [ ]:
from waveimage import WaveImage, calc_dim, mnist_reshape_32

In [ ]:
import math

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")


In [ ]:
x = mnist.train.images[0]
image = mnist_reshape_32(x, i_offset = -10, j_offset = -10)
waveImage = WaveImage(image = image)

In [ ]:
def mnist_reshape_32_buf(x):
    assert x.shape == (28 * 28,)
    image = x.reshape(28,28)
    image = np.append(np.zeros((2,28)), image, axis = 0)
    image = np.append(image, np.zeros((2,28)), axis = 0)
    image = np.append(np.zeros((32,2)), image, axis = 1)
    image = np.append(image, np.zeros((32,2)), axis = 1)
    return image

image = mnist_reshape_32_buf(mnist.train.images[0])

plt.subplot(1,2,1)
plt.imshow(image, interpolation = 'nearest', cmap = 'gray_r')
plt.subplot(1,2,2)
plt.imshow(waveImage.get_image(), interpolation = 'nearest', cmap = 'gray_r')

In [ ]:
def calcule_path(coord, axes):
    path_i = []
    path_j = []
    for h in range(0,5):
        h_opp = 5 - h
        coord_i = int(coord[0] / 2**h_opp) 
        coord_j = int(coord[1] / 2**h_opp)    
        path_i += [axes[h][coord_i]]
        path_j += [axes[h][coord_j]]
    return path_i, path_j

In [ ]:
def calcule_axes_mnist():
    axes = []
    for h in range(5):
        h_opp = 5 - h
        b_moins = -.5
        b_plus = 27.5
        intervalle = b_plus - b_moins
        coord_max = int(26.5 / 2**h_opp) + 1
        unite = max(intervalle / coord_max, 2**h_opp)
        print unite
        coords = []
        for j in range(coord_max):
            coords += [b_moins + (j+.5) * unite]
        axes += [coords]
    return axes

In [ ]:
def ajoute_coeffs(coeffs, v, h, u ):
    if h == -1 :
        coeffs[0][0,0]= v
    elif h == 0:
        coeffs[0][0,1]= v[0]
        coeffs[0][1,0] = v[1]
        coeffs[0][1,1] = v[2]
    else: 
        coeffs[h][0][u[0]][u[1]] = v[0]
        coeffs[h][1][u[0]][u[1]] = v[1]
        coeffs[h][2][u[0]][u[1]] = v[2] 



In [ ]:
def calcule_coeffs_mnist(Data, c, i):
    coeffs = init_coeffs_mnist()
    for h in range(-1,5):
        for u in U[h]:
            v = Data[c][h][u][i]
            ajoute_coeffs(coeffs, v, h, u)
    return coeffs
    


In [ ]:
def affiche_mnist(Data, c, i): 
    coeffs_aff = calcule_coeffs_mnist(Data, c, i)
    plt.imshow(-pywt.waverec2(coeffs_aff, 'haar'), interpolation='nearest', cmap='gray')



In [ ]:
def affiche_mnist_coord(Data, coeffs_aff, c, i, coord):
    coeffs_ref = calcule_coeffs_mnist(Data, c, i)
    add_coeffs_coord(coeffs_ref, coeffs_aff, coord)
    plt.imshow(-pywt.waverec2(coeffs_aff, 'haar'), interpolation='nearest', cmap='gray')
    '''axes = calcule_axes_mnist()
    path_i, path_j = calcule_path(coord, axes)
    plt.plot(path_j,path_i,'r')
    plt.plot(path_j,path_i,'r+',markersize=12)
    plt.plot(coord[1],coord[0],'+',color='r',markersize=12)
    plt.xlim([b_moins,b_plus])
    plt.ylim([b_moins,b_plus])
    plt.gca().invert_yaxis()'''



In [ ]:
def affiche_path_mnist(path_i, path_j):
    col_max = max(25, len(path_i))
    colors = plt.cm.rainbow(np.linspace(0, 1, col_max))
    b_moins = -.5
    b_plus = 27.5
    for cpt in range(len(path_i) - 1):
        plt.plot(path_j[cpt:cpt + 2],path_i[cpt:cpt + 2], color = colors[col_max - cpt - 1], linewidth= 3)
    #plt.plot(path_j,path_i,'r+',markersize=12)
    plt.xlim([b_moins,b_plus])
    plt.ylim([b_moins,b_plus])
    plt.gca().invert_yaxis()

In [ ]:
w_test = WaveImage(image = mnist_reshape_32(mnist.train.images[0]))
w_zero = WaveImage()
w_copy = w_test.copy()
        
plt.figure(figsize=(12,8))
plt.subplot(131)
plt.imshow(w_test.get_image(), interpolation='nearest', cmap='gray_r')

h_max = w_test.get_h_max()
for u in w_test.get_data()[h_max - 1]:
    w_copy.set_data(h_max - 1,u,np.zeros(3))
    w_zero.set_data(h_max - 1,u,w_test.get_data()[h_max - 1][u])
    
plt.subplot(132)
plt.imshow(w_copy.get_image(), interpolation='nearest', cmap='gray_r')

    
plt.subplot(133)
plt.imshow(w_zero.get_image(), interpolation='nearest', cmap='gray')

## Creation de la base d'apprentissage

In [ ]:
B_train = []
for i in range(len(mnist.train.images)):
    if i % 1000 == 0 :
        print i
    c = mnist.train.labels[i]
    image = mnist_reshape_32(mnist.train.images[i])
    w = WaveImage(image = image)
    data = w.get_data()
    for h in range(w.get_h_max()):
        data_h = w.get_data()[h]
        for u in data_h:
            v = data_h[u]
            B_train += [(v,(c,h,u))]                
            


## Base de test

In [ ]:
B_test = []
for i in range(len(mnist.test.images)):
    if i % 1000 == 0 :
        print i
    c = mnist.test.labels[i]
    image = mnist_reshape_32(mnist.test.images[i])
    w = WaveImage(image = image)
    data = w.get_data()
    for h in range(w.get_h_max()):
        data_h = w.get_data()[h]
        for u in data_h:
            v = data_h[u]
            B_test += [(v,(c,h,u))]                
            


## Test images simple

In [ ]:
data_visu = [[],[],[],[],[],[],[],[],[],[]]
for i in range(len(mnist.train.images)):
    c = mnist.train.labels[i]
    x = mnist.train.images[i].reshape(28,28)
    data_visu[c] += [x]


In [ ]:
image_moy_moy = np.zeros((28,28))
lin_mnist = []
for c in range(10):
    image_moy = np.mean(np.array(data_visu[c]),0)
    lin_mnist += [image_moy.reshape(28 * 28)]
    image_moy_moy += image_moy
    plt.figure(figsize=(12,8))
    plt.subplot(131)
    plt.imshow(-image_moy, interpolation='nearest', cmap='gray')
    plt.title(c)
plt.figure(figsize=(12,8))
plt.subplot(131)
plt.imshow(-image_moy_moy, interpolation='nearest', cmap='gray')
plt.title('all')

## Dictionnaire (Base d'apprentissage)

In [ ]:
Data = [[],[],[],[],[],[],[],[],[],[]]
for c in range(10):
    Data[c] = [{},{},{},{},{}] 
    
for d in B:
    v = d[0]
    c = d[1][0]
    h = d[1][1]
    u = d[1][2]
    if u in Data[c][h]:
        Data[c][h][u] += [v]
    else:
        Data[c][h][u] = [v]


## Dictionnaire (Base de test)

In [ ]:
Data_test = [[],[],[],[],[],[],[],[],[],[]]
for c in range(10):
    Data_test[c] = [{},{},{},{},{}] 
    
for d in B_test:
    v = d[0]
    c = d[1][0]
    h = d[1][1]
    u = d[1][2]
    if u in Data_test[c][h]:
        Data_test[c][h][u] += [v]
    else:
        Data_test[c][h][u] = [v]


In [ ]:
print np.mean(Data[0][0][(0,0)],0)
print np.mean(Data[0][1][(0,0)],0)
print np.mean(Data[0][1][(0,1)],0)
print np.mean(Data[0][2][(0,0)],0)
print np.mean(Data[0][2][(0,1)],0)


In [ ]:
print Data[5][0][(0,0)]

In [ ]:
c = 1
h = -1
i = 0
j = 0
len(Data[c][h][(i,j)])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize = (12,12))
ax = fig.add_subplot(111, projection='3d')

h = 0 #4
i = 0 #5
j = 0 #5

test = np.array(Data[0][h][(i,j)])
ax.scatter(test[:,0], test[:,1], test[:,2],c='r', marker = 'o')

test2 = np.array(Data[1][h][(i,j)])
ax.scatter(test2[:,0], test2[:,1], test2[:,2],c='b', marker = '^')

test3 = np.array(Data[3][h][(i,j)])
ax.scatter(test3[:,0], test3[:,1], test3[:,2],c='g', marker = 'd')

ax.view_init(20, -30)

'''for angle in range(0, 360):
    ax.view_init(30, angle)
    plt.draw()
    plt.pause(.001)'''

## Probas elementaires 

In [ ]:
mu = []
Sigma = []
rho = []
for c in range(10):
    mu += [{}]
    Sigma += [{}]
    rho += [{}]
    for h in range(-1,5):
        mu[c][h] = {}
        Sigma[c][h] = {}
        rho[c][h] = {}
        for u in U[h]:
            if u in Data[c][h]:
                data = []
                cpt = 0
                for v in Data[c][h][u]:
                    if np.linalg.norm(v) < 1e-16:
                        cpt += 1
                    else:
                        data += [v]
                if h == -1 :
                    print len(data)
                if len(data) > 1:
                    mu[c][h][u] = np.mean(data, 0) #Data[c][h][u],0)
                    Sigma[c][h][u] = np.cov(np.array(data).T) #Data[c][h][u]).T)
                    rho[c][h][u] = float(cpt) / len(Data[c][h][u])
                else:
                    mu[c][h][u] = np.zeros(3)
                    Sigma[c][h][u] = np.zeros((3,3))
                    rho[c][h][u] = 1.
            

In [ ]:
c = 0
h = -1 # 4
u = -1 #(12,12)
print mu[c][h][u]
print Sigma[c][h][u]
print rho[c][h][u]


In [ ]:
mu[0][-1][-1]

## Reconstruction


In [ ]:
for c in range(10):
    coeffs_zero = init_coeffs_mnist()
    coeffs_zero[0][0,0]= (1 - rho[c][-1][-1]) * mu[c][-1][-1]
    coeffs_zero[0][0,1]= (1 - rho[c][0][(0,0)]) * mu[c][0][(0,0)][0]
    coeffs_zero[0][1,0] = (1 - rho[c][0][(0,0)]) * mu[c][0][(0,0)][1]
    coeffs_zero[0][1,1] = (1 - rho[c][0][(0,0)]) * mu[c][0][(0,0)][2]
    for h in range(1,5):
        for u in U[h]:
            coeffs_zero[h][0][u[0]][u[1]] = (1 - rho[c][h][u]) * mu[c][h][u][0]
            coeffs_zero[h][1][u[0]][u[1]] = (1 - rho[c][h][u]) * mu[c][h][u][1]
            coeffs_zero[h][2][u[0]][u[1]] = (1 - rho[c][h][u]) * mu[c][h][u][2]

    plt.figure(figsize=(12,8))
    plt.subplot(131)
    plt.imshow(-pywt.waverec2(coeffs_zero, 'haar'), interpolation='nearest', cmap='gray')



## Test distribution

In [ ]:
fig = plt.figure(figsize = (12,12))
ax = fig.add_subplot(111, projection='3d')

c = 8
h = 3
u = (4,1)

n = 200

test2 = np.random.multivariate_normal(mu[c][h][u], Sigma[c][h][u] , n)
#test2 = np.random.multivariate_normal(mu[(c,h,u)], Sig_test * 0 , n)
#test2 = np.random.multivariate_normal(-mu[(c,h,u)], np.eye(3,3), n)
ax.scatter(test2[:,0], test2[:,1], test2[:,2],c='b', marker = '^')

#test3 = np.random.multivariate_normal(-mu[(c,h,u)], np.eye(3,3), n)
#ax.scatter(test3[:,0], test3[:,1], test3[:,2],c='g', marker = 'd')

index = np.arange(len(Data[c][h][u]))
np.random.shuffle(index)
test = np.array(Data[c][h][u])
ax.scatter(test[index[:n],0], test[index[:n],1], test[index[:n],2],c='r', marker = 'o')

v = Data_test[c][h][u][0]
ax.scatter(v[0], v[1], v[2],c='g', marker = 's')

ax.view_init(20, -30)

## Test proba

In [ ]:
from scipy.stats import multivariate_normal

c_ref = 9
#h = 0
#u = (0,0)

for i in [0]:
    pi = np.ones(10) / 10
    log_score = np.zeros(10)
    mem_pi = [pi]
    for h in range(-1,5):
        print '*************************'
        print '*********' , 'h = ' , h , '************'
        print '*************************'
        for u in U[h]:
            #print U[h]
            #u = (u_ref[1],u_ref[2])
            lik = np.zeros(10)
            for c in range(10):
                v = Data_test[c_ref][h][u][i]
                if np.linalg.norm(v) < 1e-16:
                    lik[c] = rho[c][h][u]
                else:
                    if np.linalg.norm(mu[c][h][u]) > 1e-16:
                        dist = multivariate_normal(mean = mu[c][h][u], cov = Sigma[c][h][u])
                        #print 'mean =', mu[c][h][u]
                        #print 'cov =', Sigma[c][h][u]
                        lik[c] = (1-rho[c][h][u]) * dist.pdf(v)
                    else:
                        lik[c] = 0
                lik[c] = max(lik[c],1e-16)
            print 'lik =' + str(lik) 
            log_score += np.log(lik) 
            max_log_score = max(log_score)
            print 'log_score', log_score
            print 'max(log_score) :', max_log_score
            log_score -= max_log_score
            Z = np.sum(np.exp(log_score))
            pi = np.exp(log_score)/Z
            mem_pi += [pi]
            #for c in range(10):
            #    pi[c] = lik[c]/lik.sum()
            print 'pi =' + str(pi)
            print 'max(pi) = ',max(pi)
            sum_H = - pi * np.log(lik)
            #print np.log(lik)
            print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.where(pi == max(pi))[0][0]
            print '**** H = ',sum_H.sum()
    

In [ ]:
plt.figure(figsize = (12,6))
plt.semilogy(mem_pi)
plt.ylim([1e-30,10])
#plt.legend()

## Test reconnaissance en profondeur (saccade unique)

Tests entropie effective 

On constate un biais de confirmation de l'hypothèse initiale

In [ ]:
def calcule_desc(U_tree, mem_h_u_todo):
    if U_tree == []:
        return []
    else :
        if U_tree[0] in mem_h_u_todo :
            rep = [U_tree[0]]
        else:
            rep = []
        if U_tree[1] != [] :
            rep += calcule_desc(U_tree[1], mem_h_u_todo)
        if U_tree[2] != [] :
            rep += calcule_desc(U_tree[2], mem_h_u_todo)
        if U_tree[3] != [] :
            rep += calcule_desc(U_tree[3], mem_h_u_todo)
        if U_tree[4] != [] :
            rep += calcule_desc(U_tree[4], mem_h_u_todo)    
        return rep
        
def find_desc (U_tree, pos):
    if U_tree == []:
        return None
    else :    
        if U_tree[0] == pos :
            return U_tree
        else:
            desc_1 = find_desc(U_tree[1], pos)
            if desc_1 != None:
                return desc_1
            desc_2 = find_desc(U_tree[2], pos)
            if desc_2 != None:
                return desc_2
            desc_3 = find_desc(U_tree[3], pos)
            if desc_3 != None:
                return desc_3
            desc_4 = find_desc(U_tree[4], pos)
            if desc_4 != None:
                return desc_4
    
def calcule_asc_path(h,u):
    rep = []
    for h_inf in range(h,-1,-1):
        i_inf = u[0] / (2 ** (h - h_inf))
        j_inf = u[1] / (2 ** (h - h_inf))
        rep += [(h_inf, (i_inf, j_inf))] 
    rep += [(-1,-1)]
    return rep
#test =  calcule_desc(U_tree_ref)
print calcule_asc_path(4,(8,3))
       

## Métriques


In [ ]:
def calc_lik(v,h,u):
    lik = np.zeros(10)
    for c in range(10):
        if np.linalg.norm(v) < 1e-16:
            if np.linalg.norm(mu[c][h][u]) > 1e-16:
                lik[c] = rho[c][h][u]
            else:
                lik[c] = 1                
        else:
            if np.linalg.norm(mu[c][h][u]) > 1e-16:
                dist = multivariate_normal(mean = mu[c][h][u], cov = Sigma[c][h][u])
                lik[c] = (1-rho[c][h][u]) * dist.pdf(v)
                #lik[c] =  dist.pdf(v)
            else:
                lik[c] = 0
        lik[c] = max(lik[c],1e-16)    
    return lik

def update_log_score(log_score, lik):
    #print 'lik =' + str(lik) 
    log_score += np.log(lik) 
    max_log_score = max(log_score)
    log_score -= max_log_score
    return log_score
    
def calc_pi(log_score): # TODO
    Z = np.sum(np.exp(log_score))
    pi = np.exp(log_score)/Z
    #print 'pi =' + str(pi)
    #print 'max(pi) = ',max(pi)
    return pi
    
def calc_H_plus(log_score, v_plus, h_plus, u_plus):
    lik_plus = calc_lik(v_plus, h_plus, u_plus)
    log_score_plus = update_log_score(np.copy(log_score), lik_plus)
    pi_plus = calc_pi(log_score_plus)
    sum_H = - pi_plus * np.log(pi_plus)
    #print 'sum_H : ', sum_H
    #print np.log(lik)
    #print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.where(pi == max(pi))[0][0]
    #print '**** H = ',
    return sum_H.sum()

def calc_F_plus(pi, log_score, v_plus, h_plus, u_plus):    
    lik_plus = calc_lik(v_plus, h_plus, u_plus)
    log_score_plus = update_log_score(np.copy(log_score), lik_plus)
    pi_plus = calc_pi(log_score_plus)
    sum_F = - pi_plus * (np.log(lik_plus) - np.log(pi_plus) + np.log(pi))
    #print 'sum_F : ', sum_F
    #print np.log(lik)
    #print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.where(pi == max(pi))[0][0]
    #print '**** H = ',
    return sum_F.sum()   

def calc_IB_plus(pi, log_score, v_plus, h_plus, u_plus):    
    lik_plus = calc_lik(v_plus, h_plus, u_plus)
    log_score_plus = update_log_score(np.copy(log_score), lik_plus)
    pi_plus = calc_pi(log_score_plus)
    sum_IB = pi_plus * (np.log(pi_plus) - np.log(pi))
    #print 'sum_F : ', sum_F
    #print np.log(lik)
    #print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.where(pi == max(pi))[0][0]
    #print '**** H = ',
    return sum_IB.sum()   

def calc_pi_MC_plus(pi,liste_u, nb_sample):
    nb_pos = len(liste_u)
    pi_MC_plus = np.zeros(nb_pos)
    tirage = np.random.multinomial(1,pi)
    c_MC = np.where(tirage==1)[0][0]
    for k in range(len(liste_u)) : 
        (h_plus, u_plus) = liste_u[k]
        for nb_obs in range(nb_sample):      
            if np.random.random() > rho[c_MC][h_plus][u_plus]:
                v_MC = np.random.multivariate_normal(mu[c_MC][h_plus][u_plus], Sigma[c_MC][h_plus][u_plus] , 1)[0]
            else:
                v_MC = np.zeros(3)
            lik_plus = calc_lik(v_MC, h_plus, u_plus)
            log_score_plus = update_log_score(np.zeros(10), lik_plus)
            pi_plus = calc_pi(log_score_plus)
            pi_MC_plus[k] += pi_plus[c_MC] / nb_sample
    return pi_MC_plus

def calc_pi_predictive_plus(pi,liste_u):
    nb_pos = len(liste_u)
    pi_predictive_plus = np.zeros(nb_pos)
    c_predictive = np.argmax(pi)
    for k in range(len(liste_u)) : 
        (h_plus, u_plus) = liste_u[k]
        '''if rho[c_predictive][h_plus][u_plus] < .5 :
            v_predictive = mu[c_predictive][h_plus][u_plus]
        else:
            v_predictive = np.zeros(3)
        lik_plus = calc_lik(v_predictive, h_plus, u_plus)'''
        lik_plus = lik_predictive[c_predictive][h_plus][u_plus]
        log_score_plus = update_log_score(np.zeros(10), lik_plus)
        pi_plus = calc_pi(log_score_plus)
        pi_predictive_plus[k] = pi_plus[c_predictive] 
    return pi_predictive_plus

def calc_E_plus(pi, v_plus, h_plus, u_plus):    
    lik_plus = calc_lik(v_plus, h_plus, u_plus)
    sum_E = pi * lik_plus
    #print 'sum_F : ', sum_F
    #print np.log(lik)
    #print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.where(pi == max(pi))[0][0]
    #print '**** H = ',
    return sum_E.sum()   

## Modèle prédictif

In [ ]:
lik_predictive = {}
for c in range(10):
    lik_predictive[c] = {}
    for h in range(5):
        lik_predictive[c][h] = {}
        for u in U[h]:
            log_score = np.zeros(10)
            test_pred = rho[c][h][u] < .5       
            if test_pred:
                v_predictive = mu[c][h][u]
                    #v_predictive = np.random.multivariate_normal(mu[c_predictive][h_path][u_path], Sigma[c_predictive][h_path][u_path], 1)[0]
            else:
                v_predictive = np.zeros(3)
            lik = calc_lik(v_predictive, h, u)
            lik_predictive[c][h][u] = lik

In [ ]:
pi_predictive = {}
for c in range(10):
    pi_predictive[c] = {}
    for i in range(14):
        for j in range(14):
            h = 4
            u = (i, j)
            liste_path = calcule_asc_path(h, u)
            log_score_path = np.zeros(10)
            for (h_path, u_path) in liste_path[:-1]:
                test_pred = rho[c][h_path][u_path] < .5       
                if test_pred:
                    v_predictive = mu[c][h_path][u_path]
                    #v_predictive = np.random.multivariate_normal(mu[c_predictive][h_path][u_path], Sigma[c_predictive][h_path][u_path], 1)[0]
                else:
                    v_predictive = np.zeros(3)
                lik_path = calc_lik(v_predictive, h_path, u_path)
                log_score_path = update_log_score(log_score_path, lik_path)
            pi_path = calc_pi(log_score_path)
            pi_predictive[c][(h, u)] = pi_path[c]   


In [ ]:
pi_predictive_eff = {}
for c in range(10):
    print c
    pi_predictive_eff[c] = []
    
    mem_h_u = []
    mem_h_u_todo = {}
    for u_add in U[4]:
        mem_h_u_todo[(4, u_add)] = 1
    
    while len(mem_h_u_todo) > 0 :    
        pi_predictive_plus = {}
        for (h, u) in mem_h_u_todo:
            liste_path = calcule_asc_path(h, u)
            log_score_path = np.zeros(10)
            for (h_path, u_path) in liste_path[:-1]:
                if (h_path, u_path) not in mem_h_u:
                    log_score_path = update_log_score(log_score_path, lik_predictive[c][h_path][u_path])
            pi_path = calc_pi(log_score_path)
            #print pi_path
            pi_predictive_plus[(h, u)] = pi_path[c] 
            #print h_plus, u_plus
        keys = pi_predictive_plus.keys()
        values = np.array(pi_predictive_plus.values())
        k = np.argmax(values)
        (h, u) = keys[k]
        #print len(pi_predictive_plus), len(mem_h_u), (values[k], (h, u))
        
        pi_predictive_eff[c] += [(values[k], (h, u))]
        mem_h_u_todo.pop((h,u)) 
        liste_path = calcule_asc_path(h, u)
        log_score_path = np.zeros(10)
        for (h_path, u_path) in liste_path[:-1]:
            if (h_path, u_path) not in mem_h_u:
                mem_h_u += [(h_path, u_path)]       
        
        #print 'CHOIX :', (h, u)
        
        
        

In [ ]:
print pi_predictive_eff[5]

In [ ]:
def calc_pi_predictive_sorted(): 
    pi_predictive_sorted = {}
    for c in range(10):
        pi_predictive_sorted[c] = []
        for k in pi_predictive[c]:
            pi_predictive_sorted[c] += [(pi_predictive[c][k], k)]
        pi_predictive_sorted[c] = sorted(pi_predictive_sorted[c])
    return pi_predictive_sorted

### Test fonctions MAJ

In [ ]:
c_ref = 8
#h = 0
#u = (0,0)
mem_out_final = []

for i in range(100):
    if i % 10 == 9:
        print i
    pi = np.ones(10) / 10
    log_score = np.zeros(10)
    mem_pi = [pi]
    mem_H = [np.sum(- pi * np.log(pi))]
    for h in range(-1,5):
        #print '*************************'
        #print '*********' , 'h = ' , h , '************'
        #print '*************************'
        for u in U[h]:
            v = Data_test[c_ref][h][u][i]
            lik = calc_lik(v,h,u)
            #print 'lik =' + str(lik) 
            log_score = update_log_score(log_score, lik)
            #print 'log_score =' + str(log_score) 
            pi_moins = pi
            pi = calc_pi(log_score)
            mem_pi += [pi]
            #print 'pi =' + str(pi)
            #print 'max(pi) = ',max(pi)
            F = np.sum(- pi * (np.log(lik) - np.log(pi) + np.log(pi_moins)))
            H = np.sum(- pi * np.log(pi))
            mem_H += [H]
            #print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.where(pi == max(pi))[0][0]
            #print '**** H = ', H
            #print '**** F = ', F
    out = np.argmax(pi)
    mem_out_final += [out]
    

In [ ]:
np.where(np.array(mem_out_final)==8,1,0).sum()

In [ ]:
plt.figure(figsize = (20,4))
plt.subplot(1,3,1)
affiche_mnist(Data_test, c_ref, i)
plt.subplot(1,3,2)
plt.semilogy(mem_pi)
plt.ylim([1e-30,10])
plt.title('$\pi$')
plt.subplot(1,3,3)
plt.plot(mem_H)
plt.title('Entropy')


In [ ]:
(h,u) = (4, (8, 3))
print range (h,-1,-1)

In [ ]:
print U_tree[3][1][2]

In [ ]:
c_ref = 3
print '*****', c_ref, '*****'
#h = 0
#u = (0,0)

mem_out_final = []
mem_turn = []


for i in range(1):
    pi = np.ones(10) / 10
    log_score = np.zeros(10)
    mem_pi = [pi]
    mem_pi_copy = []
    mem_H = [np.sum(- pi * np.log(pi))]
    mem_H_copy = []
    mem_h_u = []
    mem_out = []
    (h,u) = (-1,-1)
    v = Data_test[c_ref][h][u][i]     
    lik = calc_lik(v,h,u)
    print 'lik : ', lik
    log_score = update_log_score(log_score, lik)
    #print 'log_score : ', log_score
    pi = calc_pi(log_score)
    print 'pi : ', pi
    mem_pi += [pi]
    H = np.sum(- pi * np.log(pi))
    mem_H += [H]
    print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.where(pi == max(pi))[0][0]    
    
    (h, u) = (0, (0, 0))
    mem_U_tree = U_tree_ref = U_tree
    mem_h_u_todo = {(0, (0, 0)) : 1}
    TOUR = 0
    END = False
    VU = False
    SEUIL = False
    liste_u = []
    mem_u_final = []
    while END == False:
        print h, u
        if (h,u) not in mem_h_u:
            mem_h_u += [(h,u)]
            mem_h_u_todo.pop((h,u))
            v = Data_test[c_ref][h][u][i]     
            lik = calc_lik(v,h,u)
            print 'lik : ', lik
            log_score = update_log_score(log_score, lik)
            print 'log_score : ', log_score
            pi = calc_pi(log_score)
            print 'pi : ', pi
            mem_pi += [pi]
            H = np.sum(- pi * np.log(pi))
            print 'H : ', H
            mem_H += [H]
            out = np.argmax(pi)
            print 'pi[out] =', pi[out]
            print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.argmax(pi)
            VU = False
            if h == 4 and pi[out] > .999999 :
                SEUIL = True
        else :
            print h, u, 'déjà vu'
            #VU = True
            print ''
        if h < 4:
            if U_tree_ref[1] != [] :
                if U_tree_ref[1][0] not in mem_h_u :
                    mem_h_u_todo[U_tree_ref[1][0]] = 1
            if U_tree_ref[2] != [] :
                if U_tree_ref[2][0] not in mem_h_u :
                    mem_h_u_todo[U_tree_ref[2][0]] = 1
            if U_tree_ref[3] != [] :
                if U_tree_ref[3][0] not in mem_h_u :
                    mem_h_u_todo[U_tree_ref[3][0]] = 1
            if U_tree_ref[4] != [] :
                if U_tree_ref[4][0] not in mem_h_u :
                    mem_h_u_todo[U_tree_ref[4][0]] = 1
            liste_u_desc = calcule_desc(U_tree_ref, mem_h_u_todo)
            '''for i_fils in range(1,5):
                if U_tree_ref[i_fils] != [] and U_tree_ref[i_fils] not in liste_u and U_tree_ref[i_fils][0] not in mem_h_u:
                    liste_u+= [U_tree_ref[i_fils]] 
            print 'TOUR = ',TOUR, 'h = ', h, 'len(liste_u) =', len(liste_u)'''
            print 'liste_u_desc :', liste_u_desc
            for pos_test in mem_h_u:
                if pos_test in liste_u_desc:
                    liste_u_desc.remove(pos_test)
            if len(liste_u_desc) == 0 :
                print 'HUM!!', 'h = ', h, 'u = ', u
                c_in = raw_input()
                h = 0
                u = (0,0)
                mem_U_tree = U_tree_ref = U_tree
            else:
                #c = raw_input()
                '''H_plus = np.zeros(len(liste_u_desc))
                F_plus = np.zeros(len(liste_u_desc))
                E_plus = np.zeros(len(liste_u_desc))
                IB_plus = np.zeros(len(liste_u_desc))
                for k in []: #range(len(liste_u_desc)):
                    (h_plus, u_plus) = liste_u_desc[k] 
                    v_plus = Data_test[c_ref][h_plus][u_plus][i]
                    H_plus[k] = calc_H_plus(log_score, v_plus, h_plus, u_plus)
                    F_plus[k] = calc_F_plus(pi, log_score, v_plus, h_plus, u_plus)
                    E_plus[k] = calc_E_plus(pi, v_plus, h_plus, u_plus)
                    IB_plus[k] = calc_IB_plus(pi, log_score, v_plus, h_plus, u_plus)'''
                #pi_MC_plus = calc_pi_MC_plus(pi, liste_u_desc, 10)
                #print 'pi_MC_plus :', pi_MC_plus
                pi_predictive_plus = calc_pi_predictive_plus(pi,liste_u_desc)
                #print 'pi_predictive_plus :', pi_predictive_plus
                #k_plus = np.argmin(H_plus) #where(H_plus == min(H_plus))[0][0]           
                #k_plus =  np.argmax(F_plus)
                #k_plus =  np.argmin(E_plus)
                #k_plus =  np.argmax(IB_plus)
                k_plus =  np.argmax(pi_predictive_plus)
                #k_plus =  np.argmax(pi_MC_plus)
                #print 'H plus :', H_plus
                #print 'F plus :', F_plus
                #print 'k_plus :', k_plus
                (h, u) = liste_u_desc[k_plus]
                mem_U_tree = U_tree_ref                
                U_tree_ref = find_desc (U_tree_ref, liste_u_desc[k_plus]) #U_tree_ref[k_plus + 1]
                #print U_tree_ref
                print ''
        else:
            mem_u_final += [u]
            mem_pi_copy += [list(mem_pi)]
            mem_H_copy += [list(mem_H)]
            mem_out += [out]
            if TOUR == 50  or SEUIL:    
                END = True
                print '************************************'
                print '******         FINI          *******'
                print '************************************'  
                mem_out_final += [out]
                mem_turn += [TOUR]
            else:
                TOUR += 1
                print '************************************'
                print '******       TOUR    ' + str(TOUR) + '        ******'
                print '************************************'
                h = 0
                u = (0,0)
                mem_U_tree = U_tree_ref = U_tree
        

In [ ]:
np.where(np.array(mem_out_final)==3,1,0).sum()

## Affichage

In [ ]:
coeffs_aff = init_coeffs_mnist()
cpt = 0
path_i = []
path_j = []
for u in mem_u_final :
    path_i += [U[4][u][0]]
    path_j += [U[4][u][1]]
    plt.figure(figsize = (20,4))
    plt.subplot(1,4,1)
    affiche_mnist(Data_test, c_ref, i)
    affiche_path_mnist(path_i, path_j)
    plt.title(cpt)
    plt.subplot(1,4,2)    
    coord = U[4][u]  
    print 'coords :', coord
    
    affiche_mnist_coord(Data_test, coeffs_aff, c_ref, i, coord)
    affiche_path_mnist(path_i, path_j)
    plt.subplot(1,4,3)
    plt.plot(mem_pi_copy[cpt])
    plt.title('$\pi$, out = '+str(mem_out[cpt]))
    plt.ylim([0,1.05])
    plt.subplot(1,4,4)
    plt.plot(mem_H_copy[cpt])
    plt.title('Entropy')
    plt.ylim([0,2.4])
    cpt += 1

**Commentaire** : test avec `pi_MC_plus` : les saccades se focalisent sur les petits details, difficulté à atteindre un autre quart d'image que le quart initial (niveau h = 1 trop peu discriminant) ==> il faut appliquer cette technique sur la totalité de l'image.

## Predictive coding

In [ ]:
pi_predictive_sorted = calc_pi_predictive_sorted()
for c in range(10):
    image_moy = np.mean(np.array(data_visu[c]),0)
    plt.figure(figsize=(12,8))
    plt.subplot(131)
    plt.imshow(-image_moy, interpolation='nearest', cmap='gray')
    plt.title(c)
    path_i = []
    path_j = []
    image_pi = np.zeros((14,14))
    for cpt in range(10):
        #(pi_pred,(h,u)) = pi_predictive_sorted[c].pop()
        (pi_pred,(h,u)) = pi_predictive_eff[c][cpt]
        #print c,pi_pred
        path_i += [U[4][u][0]]
        path_j += [U[4][u][1]]
    affiche_path_mnist(path_i, path_j)
    
    plt.subplot(132)
    for cpt in range(196):
        (pi_pred,(h,u)) = pi_predictive_eff[c][cpt]
        image_pi[u[0],u[1]] = pi_pred
        
    
    plt.imshow(image_pi, interpolation='nearest', cmap = 'gist_heat_r', vmin = 0, vmax = 1)
    
    


## Modèle baseline

In [ ]:

mem_out_final = []
mem_turn = []

for i in range(100):
    x = mnist.test.images[i]
    scores = []
    for c in range(10):
        scores += [np.dot(x, lin_mnist[c])]
    mem_out_final += [np.argmax(scores) == mnist.test.labels[i]]
        

In [ ]:
np.sum(mem_out_final)

## Modèle prédictif rapide

In [ ]:
c_ref = 8
print '*****', c_ref, '*****'

mem_out_final = []
mem_turn = []

for i in range(1):
    pi = np.ones(10) / 10
    log_score = np.zeros(10)
    mem_pi = [pi]
    mem_pi_copy = []
    mem_H = [np.sum(- pi * np.log(pi))]
    mem_H_copy = []
    mem_h_u = []
    mem_out = []
    
    # Initialisation
    (h,u) = (-1,-1)
    v = Data_test[c_ref][h][u][i]     
    lik = calc_lik(v,h,u)
    print 'lik : ', lik
    log_score = update_log_score(log_score, lik)
    #print 'log_score : ', log_score
    pi = calc_pi(log_score)
    print 'pi : ', pi
    mem_pi += [pi]
    H = np.sum(- pi * np.log(pi))
    mem_H += [H]
    print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.where(pi == max(pi))[0][0]    
    
    (h, u) = (0, (0, 0))
    mem_h_u_todo = {(0, (0, 0)):1}
    for u_add in U[4]:
        mem_h_u_todo[(4, u_add)] = 1

    
    TOUR = 0
    END = False
    VU = False
    SEUIL = False
    liste_u = []
    mem_u_final = []

    # Premiere estimation pi (0,(0,0))
    mem_h_u += [(h,u)]
    mem_h_u_todo.pop((h,u))
    v = Data_test[c_ref][h][u][i]     
    lik = calc_lik(v,h,u)
    print 'lik : ', lik
    log_score = update_log_score(log_score, lik)
    print 'log_score : ', log_score
    pi = calc_pi(log_score)
    print 'pi : ', pi
    mem_pi += [pi]
    H = np.sum(- pi * np.log(pi))
    print 'H : ', H
    mem_H += [H]
    print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.argmax(pi)
    out = np.argmax(pi)
    
    pi_predictive_sorted = {} #calc_pi_predictive_sorted()
    for c in range(10):
        pi_predictive_sorted[c] = sorted(pi_predictive_eff[c])

    while END == False:
        
        # CHOIX
        c_predictive = np.argmax(pi)
        
        #print 'pi_predictive_sorted[', c_predictive, '] :', pi_predictive_sorted[c_predictive]
        #c = raw_input()
        (pi_plus, (h,u)) = pi_predictive_sorted[c_predictive].pop()
        
        if (h, u) in mem_h_u_todo:
            mem_h_u_todo.pop((h,u))
        else:
            while (h, u) not in mem_h_u_todo:
                (pi_plus, (h,u)) = pi_predictive_sorted[c_predictive].pop()
            mem_h_u_todo.pop((h,u))
        #print h_plus, u_plus
        print 'CHOIX : u =', u        
        
        # LECTURE
        liste_path = calcule_asc_path(h, u)
        for (h_path, u_path) in reversed(liste_path):
            if (h_path, u_path) not in mem_h_u:
                v = Data_test[c_ref][h_path][u_path][i] 
                lik_path = calc_lik(v, h_path, u_path)
                log_score = update_log_score(log_score, lik_path)
                mem_h_u += [(h_path, u_path)]
                pi = calc_pi(log_score)
                print 'pi : ', pi
                mem_pi += [pi]
                H = np.sum(- pi * np.log(pi))
                out = np.argmax(pi)
                print 'pi[out] : ', pi[out]
                print 'H : ', H
                mem_H += [H]
                
        print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.argmax(pi)
        mem_u_final += [u]
        mem_pi_copy += [list(mem_pi)]
        mem_H_copy += [list(mem_H)]
        mem_out += [out]
        #if TOUR == len(U[4]) - 1 or pi[out] > .999999:
        if TOUR == 20 or pi[out] > .9999:
        #if TOUR == 8 or pi[out] > .999:
            END = True
            print '************************************'
            print '******         FINI          *******'
            print '************************************'        
            mem_out_final += [out]
            mem_turn += [TOUR]
        else:
            TOUR += 1
            print '************************************'
            print '******       TOUR    ' + str(TOUR) + '        ******'
            print '************************************'


In [ ]:
np.where(np.array(mem_out_final)==8,1,0).sum()

In [ ]:
np.where(np.array(mem_out_final)!=1)

In [ ]:
np.mean(mem_turn)

## Modèle prédictif exact

In [ ]:
c_ref = 0
print '*****', c_ref, '*****'
#h = 0
#u = (0,0)
mem_out_final = []
mem_turn = []

for i in range(100):
    pi = np.ones(10) / 10
    log_score = np.zeros(10)
    mem_pi = [pi]
    mem_pi_copy = []
    mem_H = [np.sum(- pi * np.log(pi))]
    mem_H_copy = []
    mem_h_u = []
    mem_out = []
    
    # Initialisation
    (h,u) = (-1,-1)
    v = Data_test[c_ref][h][u][i]     
    lik = calc_lik(v,h,u)
    print 'lik : ', lik
    log_score = update_log_score(log_score, lik)
    #print 'log_score : ', log_score
    pi = calc_pi(log_score)
    print 'pi : ', pi
    mem_pi += [pi]
    H = np.sum(- pi * np.log(pi))
    mem_H += [H]
    print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.where(pi == max(pi))[0][0]    
    
    (h, u) = (0, (0, 0))
    mem_h_u_todo = {(0, (0, 0)):1}
    for u_add in U[4]:
        mem_h_u_todo[(4, u_add)] = 1

    TOUR = 0
    END = False
    VU = False
    SEUIL = False
    liste_u = []
    mem_u_final = []

    # Premiere estimation pi (0,(0,0))
    mem_h_u += [(h,u)]
    mem_h_u_todo.pop((h,u))
    v = Data_test[c_ref][h][u][i]     
    lik = calc_lik(v,h,u)
    print 'lik : ', lik
    log_score = update_log_score(log_score, lik)
    print 'log_score : ', log_score
    pi = calc_pi(log_score)
    print 'pi : ', pi
    mem_pi += [pi]
    H = np.sum(- pi * np.log(pi))
    print 'H : ', H
    mem_H += [H]
    print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.argmax(pi)
    out = np.argmax(pi)

    while END == False:
        
        # CHOIX
        pi_predictive_plus = {}
        c_predictive = np.argmax(pi)
        for (h_plus, u_plus) in mem_h_u_todo:
            liste_path = calcule_asc_path(h_plus, u_plus)
            log_score_path = np.zeros(10)
            for (h_path, u_path) in liste_path[:-1]:
                if (h_path, u_path) not in mem_h_u:
                    '''#test_pred = np.random.random() > rho[c_predictive][h_path][u_path]
                    test_pred = rho[c_predictive][h_path][u_path] < .5       
                    if test_pred:
                        v_predictive = mu[c_predictive][h_path][u_path]
                        #v_predictive = np.random.multivariate_normal(mu[c_predictive][h_path][u_path], Sigma[c_predictive][h_path][u_path], 1)[0]
                    else:
                        v_predictive = np.zeros(3)
                    lik_path = calc_lik(v_predictive, h_path, u_path)
                    log_score_path = update_log_score(log_score_path, lik_path)'''
                    log_score_path = update_log_score(log_score_path, lik_predictive[c_predictive][h_path][u_path])
            pi_path = calc_pi(log_score_path)
            pi_predictive_plus[(h_plus, u_plus)] = pi_path[c_predictive] 
            #print h_plus, u_plus
        
        keys = pi_predictive_plus.keys()
        values = np.array(pi_predictive_plus.values())
        k_next = np.argmax(values) #np.argmin(values) #
        (h, u) = keys[k_next]
        
        print 'CHOIX :', (h, u)
        
        # LECTURE
        mem_h_u_todo.pop((h,u))
        liste_path = calcule_asc_path(h, u)
        for (h_path, u_path) in reversed(liste_path):
            if (h_path, u_path) not in mem_h_u:
                v = Data_test[c_ref][h_path][u_path][i] 
                lik_path = calc_lik(v, h_path, u_path)
                log_score = update_log_score(log_score, lik_path)
                mem_h_u += [(h_path, u_path)]
                pi = calc_pi(log_score)
                print 'pi : ', pi
                mem_pi += [pi]
                H = np.sum(- pi * np.log(pi))
                out = np.argmax(pi)
                print 'pi[out] : ', pi[out]
                print 'H : ', H
                mem_H += [H]
                
        print '****', 'c :', c_ref, ', h :', h, ', u :',u, ' ---> ', np.argmax(pi)
        mem_u_final += [u]
        mem_pi_copy += [list(mem_pi)]
        mem_H_copy += [list(mem_H)]
        mem_out += [out]
        #if TOUR == 50 or (TOUR >5 and pi[out] > .99999):   
        if TOUR == len(U[4]) - 1 or pi[out] > .999999:
            END = True
            print '************************************'
            print '******         FINI          *******'
            print '************************************'   
            mem_out_final += [out]
            mem_turn += [TOUR]
        else:
            TOUR += 1
            print '************************************'
            print '******       TOUR    ' + str(TOUR) + '        ******'
            print '************************************'
        

In [ ]:
np.where(np.array(mem_out_final)==0,1,0).sum()

In [ ]:
i

In [ ]:
np.log(lik)

In [ ]:
np.random.random()

In [ ]:
lik_test = [6.47120236e+00,   1.00000000e-16,   5.20558153e-01,   1.37438396e-02,
   2.70854267e-05,   1.95244077e-01,   2.48980440e+00,   1.61746086e-04,
   3.05114329e-02,   1.48932662e-03]
log_score_test = [-44.72913926, -226.20581543,   -2.30103965,  -30.98412798,  -46.28824072,
  -32.42211479,  -30.40021047, -100.38374515,    0.,          -52.30459006]
print np.log(lik_test)

In [ ]:
log_score_plus = log_score_test + np.log(lik_test)

In [ ]:
log_score_plus

In [ ]:
b_i_inf = 2**(h-5) * i_moins

In [ ]:
a = np.random.multinomial(1,[.25, .25, .25, .25])
np.where(a==1)[0][0]

In [ ]:
l = range(10)
print l.pop(), l

In [ ]:
x = range(5)
y = sorted(x, reverse = True)
y[0] = 2
print x
print y

In [ ]:
colors = plt.cm.rainbow(np.linspace(0, 1, len(path_i)))

In [ ]:
plt.plot(np.random.rand(10), color = colors[1])

In [ ]:
len(path_i)